# 1. Data Ingestion

## a. Importing Libraries

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

import yfinance as yf

from datetime import datetime
import requests
import concurrent.futures
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from numpy.polynomial.polynomial import Polynomial
import matplotlib.pyplot as plt
from scipy.stats import norm
from google.cloud import bigquery

drive_folder = "/content/drive/MyDrive/Grad Pricing simulations Data Prep Runs 2/"
os.makedirs(drive_folder, exist_ok=True)

Mounted at /content/drive


## b. Data Preparation

In [ ]:
def get_stock_data(tickers, start_date, end_date):
    all_data = []
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date, end=end_date)

        if data.empty:
            print(f"Warning: No data found for {ticker}")
            continue

        data['Ticker'] = ticker
        all_data.append(data)

    if not all_data:
        raise ValueError("No valid stock data retrieved for tickers!")

    historical_data_combined = pd.concat(all_data)
    historical_data_combined.reset_index(inplace=True)

    return historical_data_combined

def add_shock_events(df):
    """
    Label shock events in your stock_data dataframe.
    Adds a 'shock_event' column based on date ranges.
    """

    df['shock_event'] = 'None'

    # --- Volmageddon Crash ---
    df.loc[(df['Date'] >= '2018-02-01') & (df['Date'] <= '2018-03-31'), 'shock_event'] = 'volmageddon'

    # --- 2018 Q4 Sell-Off ---
    df.loc[(df['Date'] >= '2018-10-01') & (df['Date'] <= '2018-12-31'), 'shock_event'] = '2018_q4_selloff'

    # --- COVID Crash ---
    df.loc[(df['Date'] >= '2020-02-15') & (df['Date'] <= '2020-05-01'), 'shock_event'] = 'covid_crash'

    # --- Ukraine Crisis ---
    df.loc[(df['Date'] >= '2022-02-01') & (df['Date'] <= '2022-05-01'), 'shock_event'] = 'ukraine_crisis'

    # --- Fed Hikes / Inflation Surge ---
    df.loc[(df['Date'] >= '2022-06-01') & (df['Date'] <= '2022-12-31'), 'shock_event'] = 'fed_hike_inflation'

    # --- SVB / Regional Banking Crisis ---
    df.loc[(df['Date'] >= '2023-03-01') & (df['Date'] <= '2023-06-01'), 'shock_event'] = 'svb_crisis'

    return df

def get_dividend_yield(ticker):
    """ Fetch dividend yield from Yahoo Finance """
    stock = yf.Ticker(ticker)
    return stock.info.get("dividendYield", 0) or 0.0

def get_risk_free_rate():
    """ Fetch the latest 6-month U.S. Treasury yield as a proxy for the risk-free rate """
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": "DGS6MO",  # 6-month US Treasury yield
        "api_key": "ef17cd670af1026241ee7fb83d925738",  #api key from FRED
        "file_type": "json",
        "limit": 5,  # we fetch multiple observations to ensure we get the latest one
        "sort_order": "desc"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if "observations" in data and data["observations"]:
            latest_observation = max(data["observations"], key=lambda x: x["date"])  # get the latest date
            rate = float(latest_observation["value"]) / 100
            print("Latest Risk-Free Rate:", rate)  # debug: ensures correct value
            return rate
    return 0.05  # default fallback (5%)


def calculate_rolling_volatility(stock_data, window=30):
    """
    Calculate rolling annualized volatility for multiple tickers using historical data.
    Uses `Close` price if `Adj Close` is missing.
    """
    if stock_data.empty or 'Close' not in stock_data:
        print("Error: Stock data is missing or 'Close' not found. Returning default volatility (20%).")
        return pd.Series(0.20, index=stock_data['Ticker'].unique())

    if 'Ticker' not in stock_data:
        print("Error: Missing 'Ticker' column. Cannot compute for multiple tickers.")
        return pd.Series(0.20)

    results = {}

    for ticker in stock_data['Ticker'].unique():
        ticker_data = stock_data[stock_data['Ticker'] == ticker].copy()

        if len(ticker_data) < window:
            print(f"Warning: Not enough data for {ticker} ({len(ticker_data)} days). Using default volatility.")
            results[ticker] = 0.20
            continue

        ticker_data['Price'] = ticker_data['Close']
        ticker_data['Returns'] = ticker_data['Price'].pct_change()

        if ticker_data['Returns'].isna().all():
            print(f"Error: No valid returns for {ticker}. Using default volatility.")
            results[ticker] = 0.20
            continue

        rolling_volatility = ticker_data['Returns'].rolling(window).std() * np.sqrt(252)

        if rolling_volatility.dropna().empty:
            print(f"Error: Rolling volatility calculation resulted in NaN for {ticker}. Using default.")
            results[ticker] = 0.20
            continue

        results[ticker] = rolling_volatility.dropna().iloc[-1]

    return pd.Series(results)


def calculate_time_to_expiry(expiry_date):
    """ Compute time to expiry in years """
    today = datetime.today()
    expiry = datetime.strptime(expiry_date, "%Y-%m-%d")
    return (expiry - today).days / 365.0

def get_sector(ticker):
    return yf.Ticker(ticker).info.get('sector', 'Unknown')

In [ ]:
technology = ["AAPL", "MSFT", "NVDA", "AMD", "INTC", "QCOM", "AVGO", "ORCL", "CRM", "TXN", "ADBE", "MU", "AMAT", "IBM", "CSCO"]
financials = ["JPM", "BAC", "GS", "WFC", "MS", "C", "AXP", "USB", "SCHW", "PNC", "BK", "MET"]
communication_services = ["GOOGL", "META", "DIS", "NFLX", "T", "VZ", "PARA"]
consumer_staples = ["KO", "PG", "CL", "MO", "KHC", "KR"]
healthcare = ["UNH", "CVS", "MDT", "LLY", "AMGN", "GILD", "JNJ", "PFE", "MRK", "ABBV", "BMY"]
consumer_discretionary = ["TSLA", "HD", "GM", "F", "TGT", "MAR", "NKE", "MCD", "SBUX", "LOW", "WMT", "COST"]
energy = ["XOM", "CVX", "COP", "SLB", "HAL", "EOG", "PSX", "VLO", "MPC", "OXY"]
industrials = ["NOC", "UPS", "UNP", "RTX", "CAT", "BA", "LMT", "GE", "DE", "HON"]
utilities = ["NEE", "SO", "DUK", "EXC", "AEP"]
real_estate = ["PLD", "AMT", "SPG"]
materials = ["LIN", "NEM", "DD", "FCX", "APD", "ALB"]

In [ ]:
start_date = "2021-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

stock_data = get_stock_data(consumer_discretionary, start_date, end_date)

stock_data = add_shock_events(stock_data)

stock_data['Sector'] = stock_data['Ticker'].apply(get_sector)

stock_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,shock_event,Sector
0,2021-01-04 00:00:00-05:00,239.820007,248.163330,239.063339,243.256668,145914600,0.0,0.0,TSLA,None,Consumer Discretionary
1,2021-01-05 00:00:00-05:00,241.220001,246.946671,239.733337,245.036667,96735600,0.0,0.0,TSLA,None,Consumer Discretionary
2,2021-01-06 00:00:00-05:00,252.830002,258.000000,249.699997,251.993332,134100000,0.0,0.0,TSLA,None,Consumer Discretionary
3,2021-01-07 00:00:00-05:00,259.209991,272.329987,258.399994,272.013336,154496700,0.0,0.0,TSLA,None,Consumer Discretionary
4,2021-01-08 00:00:00-05:00,285.333344,294.829987,279.463318,293.339996,225166500,0.0,0.0,TSLA,None,Consumer Discretionary
...,...,...,...,...,...,...,...,...,...,...,...
12931,2025-04-11 00:00:00-04:00,963.960022,970.210022,943.159973,963.409973,2721800,0.0,0.0,COST,None,Consumer Discretionary
12932,2025-04-14 00:00:00-04:00,970.000000,987.130005,965.000000,979.320007,2219900,0.0,0.0,COST,None,Consumer Discretionary
12933,2025-04-15 00:00:00-04:00,985.349976,994.000000,974.250000,976.919983,1788200,0.0,0.0,COST,None,Consumer Discretionary
12934,2025-04-16 00:00:00-04:00,972.130005,978.650024,959.239990,967.750000,2531700,0.0,0.0,COST,None,Consumer Discretionary


# 2. Feature Engineering

## a. RSI, MACD, Greeks calculation

In [ ]:
# RSI Calculation
def rsi_calc(series, window=14):
    delta = series.diff()
    gain = delta.clip(lower=0).rolling(window=window).mean()
    loss = -delta.clip(upper=0).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# MACD (returns two series: macd, signal)
def get_macd(series, short=12, long=26, signal=9):
    short_ema = series.ewm(span=short, adjust=False).mean()
    long_ema = series.ewm(span=long, adjust=False).mean()
    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

In [ ]:
def calculate_baw_price(S, K, T, r, sigma, q=0.0, option_type="put"):
    """
    Helper: Price only, no Greeks.
    """
    q = 0.0
    epsilon = 1e-5

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    euro_put = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    M = 2 * r / sigma**2
    N = 2 * (r - q) / sigma**2
    q2 = (-(N - 1) - np.sqrt((N - 1)**2 + 4 * M)) / 2

    if np.abs(q2) < epsilon:
        q2 = -epsilon

    denom = 1 - 1 / q2
    if np.abs(denom) < epsilon:
        denom = -epsilon

    S_critical = K / denom

    if S <= S_critical:
        return K - S

    A2 = (1 - np.exp(- (r - q) * T) * norm.cdf(-d1)) * S_critical / q2
    return euro_put + A2 * (S / S_critical) ** q2

def finite_diff(param, S, K, T, r, sigma, option_type='put', h=1e-4):
    """
    Central finite difference approximation for first derivatives
    """
    if param == 'S':
        f_plus = calculate_baw_price(S + h, K, T, r, sigma, option_type)
        f_minus = calculate_baw_price(S - h, K, T, r, sigma, option_type)
    elif param == 'sigma':
        f_plus = calculate_baw_price(S, K, T, r, sigma + h, option_type)
        f_minus = calculate_baw_price(S, K, T, r, sigma - h, option_type)
    elif param == 'T':
        f_plus = calculate_baw_price(S, K, T + h, r, sigma, option_type)
        f_minus = calculate_baw_price(S, K, T - h, r, sigma, option_type)
    elif param == 'r':
        f_plus = calculate_baw_price(S, K, T, r + h, sigma, option_type)
        f_minus = calculate_baw_price(S, K, T, r - h, sigma, option_type)
    else:
        raise ValueError("Invalid param name for finite_diff")

    return (f_plus - f_minus) / (2 * h)

def second_finite_diff(param, S, K, T, r, sigma, option_type='put', h=1e-4):
    """
    Central finite difference approximation for second derivatives (gamma)
    """
    if param == 'S':
        f_plus = calculate_baw_price(S + h, K, T, r, sigma, option_type)
        f = calculate_baw_price(S, K, T, r, sigma, option_type)
        f_minus = calculate_baw_price(S - h, K, T, r, sigma, option_type)
    else:
        raise ValueError("Invalid param name for second_finite_diff")

    return (f_plus - 2 * f + f_minus) / (h ** 2)

In [ ]:
def calculate_greeks(S, K, T, r, sigma, q=0.0, option_type="call"):
    """
    Barone-Adesi Whaley approximation for American options (put only).
    Black-Scholes European for calls without dividends.

    Returns price + Greeks (delta, gamma, vega, theta, rho)
    """

    # ensure q is zero for no dividends
    q = 0.0
    epsilon = 1e-5

    # handle zero time or zero volatility edge cases
    if T <= 0 or sigma <= 0:
        intrinsic = max(0, S - K) if option_type == "call" else max(0, K - S)
        return {
            'price': intrinsic,
            'delta': 1.0 if intrinsic > 0 and option_type == "call" else (-1.0 if intrinsic > 0 else 0.0),
            'gamma': 0.0,
            'vega': 0.0,
            'theta': 0.0,
            'rho': 0.0
        }

    # Black-Scholes Calculations
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    # Call Options (No Dividends = EU logic)
    if option_type == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        delta = norm.cdf(d1)
        gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
        vega = S * norm.pdf(d1) * np.sqrt(T)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
                 - r * K * np.exp(-r * T) * norm.cdf(d2))
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)

        return {
            'price': price,
            'delta': delta,
            'gamma': gamma,
            'vega': vega / 100,   # Return per 1% vol change if you prefer
            'theta': theta / 365, # Return per day
            'rho': rho / 100      # Return per 1% rate change
        }

    # Put Options (American)
    # Barone-Adesi Whaley approximation

    M = 2 * r / sigma**2
    N = 2 * (r - q) / sigma**2
    q2 = (-(N - 1) - np.sqrt((N - 1)**2 + 4 * M)) / 2

    if np.abs(q2) < epsilon:
        q2 = -epsilon

    denom = 1 - 1 / q2
    if np.abs(denom) < epsilon:
        denom = -epsilon

    S_critical = K / denom

    # European put as base
    euro_put = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    if S <= S_critical:
        price = K - S
    else:
        A2 = (1 - np.exp(- (r - q) * T) * norm.cdf(-d1)) * S_critical / q2
        price = euro_put + A2 * (S / S_critical) ** q2

    # Greeks for American Put using finite differences on BAW price
    delta = finite_diff('S', S, K, T, r, sigma, option_type='put')
    gamma = second_finite_diff('S', S, K, T, r, sigma, option_type='put')
    vega = finite_diff('sigma', S, K, T, r, sigma, option_type='put')
    theta = finite_diff('T', S, K, T, r, sigma, option_type='put')
    rho = finite_diff('r', S, K, T, r, sigma, option_type='put')

    return {
        'price': price,
        'delta': delta,
        'gamma': gamma,
        'vega': vega / 100,   # optional scaling
        'theta': theta / 365, # per day decay
        'rho': rho / 100
    }

In [ ]:
# Sort data to apply windows correctly
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data.sort_values(by=['Ticker', 'Date'], inplace=True)

# Calculate returns
stock_data['Return'] = stock_data.groupby('Ticker')['Close'].transform(lambda x: x.pct_change())

# Rolling Volatility (30 days)
stock_data['RollingVol_30d'] = stock_data.groupby('Ticker')['Return'].transform(lambda x: x.rolling(window=30).std() * np.sqrt(252))

# Skewness and Kurtosis on Returns
stock_data['Skewness_30d'] = stock_data.groupby('Ticker')['Return'].transform(lambda x: x.rolling(window=30).apply(lambda y: pd.Series(y).skew()))
stock_data['Kurtosis_30d'] = stock_data.groupby('Ticker')['Return'].transform(lambda x: x.rolling(window=30).apply(lambda y: pd.Series(y).kurt()))

# RSI (14 days)
stock_data['RSI_14'] = stock_data.groupby('Ticker')['Close'].transform(lambda x: rsi_calc(x))

# Apply MACD to each ticker group
stock_data['MACD'] = stock_data.groupby('Ticker')['Close'].transform(lambda x: get_macd(x)[0])
stock_data['MACD_Signal'] = stock_data.groupby('Ticker')['Close'].transform(lambda x: get_macd(x)[1])

# Volume Based Features

# Rolling averages
stock_data['Avg_Volume_20d'] = stock_data.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(window=20, min_periods=1).mean())
stock_data['Avg_Volume_30d'] = stock_data.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
stock_data['Avg_Volume_60d'] = stock_data.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(window=60, min_periods=1).mean())

# Relative volume
stock_data['Relative_Volume_20d'] = stock_data['Volume'] / stock_data['Avg_Volume_20d']

# Volume Change %
stock_data['Volume_Change_1d'] = stock_data.groupby('Ticker')['Volume'].transform(lambda x: x.pct_change())
stock_data['Volume_Change_5d'] = stock_data.groupby('Ticker')['Volume'].transform(lambda x: x.pct_change(periods=5))


## b. Binomial Tree (for target variable simulation)

In [ ]:
def fast_binomial_tree_american_option(S0, K, T, r, sigma, N=100, option_type="call"):
    dt = T / N
    discount = np.exp(-r * dt)

    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    ST = S0 * (u ** np.arange(N, -1, -1)) * (d ** np.arange(0, N + 1, 1))

    if option_type == "call":
        option_values = np.maximum(ST - K, 0)
    else:
        option_values = np.maximum(K - ST, 0)

    for i in range(N - 1, -1, -1):
        option_values = discount * (p * option_values[:-1] + (1 - p) * option_values[1:])
        ST = S0 * (u ** np.arange(i, -1, -1)) * (d ** np.arange(0, i + 1, 1))

        if option_type == "call":
            exercise = np.maximum(ST - K, 0)
        else:
            exercise = np.maximum(K - ST, 0)

        option_values = np.maximum(option_values, exercise)

    return option_values[0]

## c. Market Simulation

In [ ]:
# Helper Functions for Each Enhancement

def apply_volatility_clustering(base_sigma, vol_of_vol=0.05, shock_prob=0.05):
    """
    Simulate clustered volatility regimes (vol shocks).
    """
    shock = np.random.rand() < shock_prob
    if shock:
        # Apply a volatility shock (increase or decrease)
        vol_shock = 1 + np.random.uniform(-vol_of_vol, vol_of_vol)
        return max(base_sigma * vol_shock, 0.01)  # Avoid zero or negative vol
    return base_sigma

# Simulate Risk-Free Rate

def simulate_risk_free_rate(base_r, r_volatility=0.005):
    """
    Simulate stochastic risk-free rate with small random volatility.
    """
    simulated_r = base_r + np.random.normal(0, r_volatility)
    return max(simulated_r, 0.0)

# Estimate Early Exercise Probability

# def estimate_early_exercise_probability(S0, K, T, sigma, option_type, dividend_yield): -- OLD
#     """
#     Simple early exercise probability heuristic.
#     """
#     moneyness = S0 / K
#     if option_type == 'put':
#         if moneyness < 0.8 and T < 0.1:
#             return 0.8
#         elif moneyness < 0.9 and T < 0.2:
#             return 0.5
#         else:
#             return 0.1
#     elif option_type == 'call':
#         if dividend_yield > 0.03 and moneyness > 1.1 and T < 0.2:
#             return 0.7
#         elif moneyness > 1.2 and T < 0.1:
#             return 0.5
#         else:
#             return 0.1
#     return 0.0

def estimate_early_exercise_probability(S0, K, T, base_r, sigma, option_type, RSI, MACD, Volume, Avg_Volume_20d, dividend_yield):

    # feature transformations
    moneyness = S0 / K
    log_moneyness = np.log(moneyness + 1e-6)
    rel_volume = Volume / (Avg_Volume_20d + 1e-6)
    log_rel_volume = np.log(rel_volume + 1e-3)
    RSI_score = (RSI - 50) / 50     # range ~[-1, 1]
    MACD_score = np.tanh(MACD)      # range ~[-1, 1]


    # smooth scoring logic (weights tuned heuristically)
    if option_type == 'call':
        score = (
            + 6.0 * (moneyness - 1.1) +
            + 8.0 * (0.2 - T) +
            + 7.0 * dividend_yield +
            + 2.0 * RSI_score +
            + 1.5 * MACD_score +
            + 1.0 * log_rel_volume -
            4.0 * sigma
        )
    else:  # put
        score = (
            + 8.0 * (0.9 - moneyness) +
            + 10.0 * (0.2 - T) +
            + 1.5 * RSI_score +
            + 0.8 * log_rel_volume -
            3.0 * sigma
        )

    # final probability output
    prob = 1 / (1 + np.exp(-score))
    jitter = np.random.normal(0, 0.015)
    return float(np.clip(prob + jitter, 0, 1))


def apply_iv_surface_adjustment(sigma, moneyness, skew_factor=0.2):
    """
    Simulate an implied volatility surface adjustment based on moneyness.
    """
    if moneyness < 0.9:  # Deep OTM puts = higher IV
        sigma_adj = sigma * (1 + skew_factor)
    elif moneyness > 1.1:  # Deep ITM calls = lower IV
        sigma_adj = sigma * (1 - skew_factor / 2)
    else:
        sigma_adj = sigma
    return max(sigma_adj, 0.01)

def apply_jump_diffusion(S0, jump_prob=0.01, jump_mean=0, jump_std=0.05):
    if np.random.rand() < jump_prob:
        # log-normal shock factor
        jump_factor = np.exp(np.random.normal(jump_mean, jump_std))
        return S0 * jump_factor
    return S0

def simulate_bid_ask_spread(base_price, spread_percent=0.01):
    spread = base_price * spread_percent
    bid = base_price - spread / 2
    ask = base_price + spread / 2
    # randomly choose to simulate taking bid/ask/mid
    return np.random.choice([bid, ask, base_price])

def apply_risk_aversion(base_price, risk_aversion=0.05):
    adjustment = np.random.normal(0, risk_aversion)
    return base_price * (1 + adjustment)

def apply_earnings_volatility(sigma, T, earnings_date_proximity_threshold=0.02):
    if T < earnings_date_proximity_threshold:
        return sigma * 1.5  # spike IV near earnings
    return sigma

def apply_iv_term_structure(sigma, T, base_curve_slope=-0.1):
    return sigma * (1 + base_curve_slope * T)

def apply_market_regime():
    """
    Randomly select a regime and return skew and vol adjustments.
    """
    regime = np.random.choice(['bull', 'bear', 'neutral'], p=[0.4, 0.3, 0.3])
    if regime == 'bull':
        skew_factor = -0.1
        vol_multiplier = 0.9
    elif regime == 'bear':
        skew_factor = 0.2
        vol_multiplier = 1.2
    else:  # neutral
        skew_factor = 0.0
        vol_multiplier = 1.0
    return regime, skew_factor, vol_multiplier

def apply_shock_adjustments(row, sigma_final, r_dynamic, liquidity_penalty=0.0, jump_probability=0.01):
    """
    Adjust volatility, risk-free rate, liquidity penalty, and jump probability based on shock events.
    """

    # default baseline before applying shocks
    event = row.get('shock_event', 'None')
    sector = row.get('Sector', 'Unknown')

    # --- Volmageddon (Feb-Mar 2018) ---
    if event == 'volmageddon':
        sigma_final *= 2.0
        liquidity_penalty += 0.05
        jump_probability += 0.05

    # --- 2018 Q4 Selloff ---
    elif event == '2018_q4_selloff':
        sigma_final *= 1.8
        liquidity_penalty += 0.05
        jump_probability += 0.03

        # Extra pain for Tech during 2018 Q4
        if sector == 'Technology':
            sigma_final *= 1.3
            jump_probability += 0.02

    # --- COVID Crash ---
    elif event == 'covid_crash':
        sigma_final *= 2.5
        liquidity_penalty += 0.2
        jump_probability += 0.1

        if sector in ['Financials', 'Consumer Discretionary']:
            sigma_final *= 1.4
            jump_probability += 0.05

    # --- Ukraine Crisis ---
    elif event == 'ukraine_crisis':
        sigma_final *= 1.8
        liquidity_penalty += 0.05
        jump_probability += 0.03

        # energy stocks benefit, others suffer
        if sector == 'Energy':
            sigma_final *= 0.8  # less volatility for energy producers
        elif sector in ['Industrials', 'Financials']:
            sigma_final *= 1.2
            jump_probability += 0.02

    # --- Fed Hikes / Inflation Surge ---
    elif event == 'fed_hike_inflation':
        sigma_final *= 1.5
        liquidity_penalty += 0.03
        jump_probability += 0.02

        if sector == 'Technology':
            sigma_final *= 1.3
            liquidity_penalty += 0.05
        elif sector == 'Financials':
            sigma_final *= 1.1  # financials benefit slightly from higher rates

    # --- SVB Crisis (Regional Banks) ---
    elif event == 'svb_crisis':
        sigma_final *= 1.7
        liquidity_penalty += 0.05
        jump_probability += 0.05

        if sector == 'Financials':
            sigma_final *= 1.5
            jump_probability += 0.05

    return sigma_final, r_dynamic, liquidity_penalty, jump_probability


## d. Generate Realistic Synthetic Option Price (All Enhancements)

In [ ]:
def generate_realistic_synthetic_price(
    S0, K, T, base_r, base_sigma, option_type,
    RSI, MACD, Volume, Avg_Volume_20d,
    dividend_yield=0.02,
    row=None
):
    """
    Generate a realistic synthetic option price with enhancements for realism.
    """

    # Market Regime
    regime, skew_factor, vol_multiplier = apply_market_regime()

    # Volatility Clustering
    sigma_clustered = apply_volatility_clustering(base_sigma)

    # IV Smile/Skew + Market Regime
    moneyness = S0 / K
    sigma_iv = apply_iv_surface_adjustment(sigma_clustered, moneyness, skew_factor)

    # Earnings Event Vol Spike
    sigma_earnings = apply_earnings_volatility(sigma_iv, T)

    # Term Structure
    sigma_final = apply_iv_term_structure(sigma_earnings, T)

    # Market Regime Vol Multiplier
    sigma_final *= vol_multiplier

    # Risk-Free Rate Dynamics
    r_dynamic = simulate_risk_free_rate(base_r)

    # Liquidity Penalty and Jump Probability Defaults
    liquidity_penalty = 0.0
    jump_probability = 0.01  # baseline

    # Apply Shock Adjustments
    if row is not None:
        sigma_final, r_dynamic, liquidity_penalty_shock, jump_probability = apply_shock_adjustments(
            row,
            sigma_final,
            r_dynamic
        )
        liquidity_penalty += liquidity_penalty_shock

    # Jump Diffusion on Underlying
    S0_jump = apply_jump_diffusion(S0, jump_prob=jump_probability)

    early_ex_prob = estimate_early_exercise_probability(
        S0_jump, K, T, base_r, sigma_final, option_type, RSI, MACD, Volume, Avg_Volume_20d, dividend_yield
    )

    # Base Price Calculation (Binomial Tree)
    base_price = fast_binomial_tree_american_option(
        S0_jump, K, T, r_dynamic, sigma_final, N=100, option_type=option_type
    )

    # Sentiment Adjustments (RSI / MACD)
    sentiment_adjustment = 0.0
    if RSI > 70 and option_type == 'call':
        sentiment_adjustment += base_price * 0.10
    elif RSI < 30 and option_type == 'put':
        sentiment_adjustment += base_price * 0.10

    macd_adjustment = MACD * 0.05 * base_price

    # Volume-Based Pricing Noise + Liquidity Penalty
    volume_ratio = Volume / Avg_Volume_20d if Avg_Volume_20d != 0 else 1
    if volume_ratio < 0.5:
        liquidity_penalty += 0.05  # combine shock + thin volume
        liquidity_noise = np.random.normal(0, 0.05 * base_price)
    else:
        liquidity_noise = np.random.normal(0, 0.01 * base_price)

    liquidity_penalty_amount = base_price * liquidity_penalty

    # Risk Aversion Adjustment
    risk_adjusted_price = apply_risk_aversion(base_price)

    # Bid-Ask Spread Simulation
    bid_ask_price = simulate_bid_ask_spread(risk_adjusted_price)

    # Early Exercise Discount
    exercise_discount = base_price * early_ex_prob * 0.1

    # Aggregate Final Adjustments
    adjusted_price = (
        bid_ask_price +
        sentiment_adjustment +
        macd_adjustment -
        exercise_discount -
        liquidity_penalty_amount +
        liquidity_noise
    )

    # Realistic Price Floor
    adjusted_price = max(adjusted_price, 0.05)

    return adjusted_price, {
        'simulated_r': r_dynamic,
        'sigma_final': sigma_final,
        'early_ex_prob': early_ex_prob,
        'regime': regime
    }

## e. Generating the scenarios

In [ ]:
def generate_option_scenarios(row, strike_multipliers, expiry_days, r_list):
    """
    Generates option scenarios for a single row of stock_data.
    Includes volume-based features, shock events, sector tags, and enhanced synthetic pricing logic.
    """
    scenarios = []

    S0 = row['Close']
    sigma = row['RollingVol_30d']
    RSI = row['RSI_14']
    MACD = row['MACD']
    Skewness = row['Skewness_30d']
    Kurtosis = row['Kurtosis_30d']
    Volume = row['Volume']
    Avg_Volume_20d = row.get('Avg_Volume_20d', Volume)

    Relative_Volume = row.get('Relative_Volume', 1.0)
    Volume_Change_1d = row.get('Volume_Change_1d', 0.0)
    Volume_Change_5d = row.get('Volume_Change_5d', 0.0)

    ticker = row['Ticker']
    close = row['Close']
    date = row['Date']
    shock_event = row.get('shock_event', 'None')
    sector = row.get('Sector', 'Unknown')

    dividend_yield = row.get('Dividends', 0.02)

    if pd.isna(sigma):
        return []

    # Iterate through the scenarios for r, strike multipliers, expiries, and option types
    for base_r in r_list:
        for multiplier in strike_multipliers:
            K = S0 * multiplier
            moneyness = S0 / K
            log_moneyness = np.log(moneyness)

            for days_to_expiry in expiry_days:
                T = days_to_expiry / 365.0

                for option_type_int in [1, 0]:  # 1 = call, 0 = put
                    option_str = 'call' if option_type_int == 1 else 'put'

                    # Generate realistic synthetic option price and metadata
                    adjusted_price, extras = generate_realistic_synthetic_price(
                        S0, K, T, base_r, sigma, option_str,
                        RSI, MACD, Volume, Avg_Volume_20d,
                        dividend_yield,
                        row=row  #  pass for shock_event/sector impact
                    )

                    # Use the updated sigma if it was modified in generation
                    updated_sigma = extras.get('sigma_final', sigma)

                    # Calculate Greeks with updated sigma and simulated r
                    greeks = calculate_greeks(
                        S0, K, T, extras['simulated_r'], updated_sigma, option_type=option_str
                    )

                    scenario = {
                        # Basic info
                        'Date': date,
                        'Ticker': ticker,
                        'Sector': sector,
                        'shock_event': shock_event,
                        'Close': close,

                        # Market conditions
                        'S0': S0,
                        'K': K,
                        'T': T,
                        'sigma': updated_sigma,
                        'r': extras['simulated_r'],
                        'moneyness': moneyness,
                        'log_moneyness': log_moneyness,

                        # Technical indicators
                        'RSI': RSI,
                        'MACD': MACD,
                        'Skewness': Skewness,
                        'Kurtosis': Kurtosis,

                        # Volume metrics
                        'Relative_Volume': Relative_Volume,
                        'Volume_Change_1d': Volume_Change_1d,
                        'Volume_Change_5d': Volume_Change_5d,

                        # Greeks
                        'delta': greeks['delta'],
                        'gamma': greeks['gamma'],
                        'vega': greeks['vega'],
                        'theta': greeks['theta'],
                        'rho': greeks['rho'],

                        # Option specifics
                        'early_exercise_prob': extras['early_ex_prob'],
                        'option_type': option_type_int,  # 1 = call, 0 = put
                        'price': adjusted_price
                    }

                    scenarios.append(scenario)

    return scenarios

## e. Feature Engineering and target simulation call

In [ ]:
r = get_risk_free_rate()

strike_multipliers = [0.8, 0.9, 1.0, 1.1, 1.2]
expiry_days = [7, 30, 90, 180, 365]
r_list = [max(0, r - 0.01), r - 0.003, r, r + 0.003, r + 0.01]

Latest Risk-Free Rate: 0.042199999999999994


In [ ]:
def process_row(row_dict):
    return generate_option_scenarios(
        row_dict,
        strike_multipliers=strike_multipliers,
        expiry_days=expiry_days,
        r_list=r_list
    )

# Convert DataFrame rows to dicts (more efficient for parallel)
row_dicts = stock_data.to_dict(orient='records')

option_scenarios = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(tqdm(
        executor.map(process_row, row_dicts),
        total=len(row_dicts),
        desc='Generating Scenarios (Parallel)'
    ))

    for scenario_list in results:
        option_scenarios.extend(scenario_list)

# Convert the list of scenarios to a DataFrame
option_scenarios_df = pd.DataFrame(option_scenarios)

Generating Scenarios (Parallel):  47%|████▋     | 6100/12936 [14:55<16:44,  6.81it/s]
Process ForkProcess-19:
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.11/concurrent/futures/process.py", line 249, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
          ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 430, in _recv_bytes
    buf = self._recv(4)
          ^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocess

KeyboardInterrupt: 

In [ ]:
#1-3
parquet_filename = os.path.join(drive_folder, f'scenarios_batch_consumer_discretionary.parquet')
option_scenarios_df.to_parquet(parquet_filename, index=False, compression='snappy')